# システム統合４（有限オートマトン＋機械学習言語理解）
それでは、これまでに実装してきたモジュールを統合して音声対話システムを作成してみましょう。

- 仕様
    - 音声認識：Google（ストリーミング、オリジナルVAD）
    - 音声合成：Google
    - 言語理解：機械学習（天気案内のスロット値推定）
    - 対話管理：有限オートマトン

In [2]:
# 必要なラブラリを読み込む
from __future__ import division

import re
import sys, os

import numpy as np
import math
import struct

import pyaudio
from six.moves import queue

from pydub import AudioSegment
from pydub.playback import play

from google.cloud import speech

# 実装したライブラリを読み込む
from asr_google_streaming_vad import GoogleStreamingASR, MicrophoneStream
from tts_google import GoogleTextToSpeech
from dm_fst_weather import DmFst
from slu_ml import SluML

C:\Users\inoue\anaconda3\envs\ai-kouza\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# 音声認識クラスのパラメータ
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

# 音声合成の初期化
tts = GoogleTextToSpeech()

# 言語理解の初期化
slu_parser = SluML()

In [5]:
# 対話管理の初期化
dm = DmFst()

# 初期状態の発話
system_utterance = dm.get_system_utterance()
tts.generate(system_utterance)
print("システム： " + system_utterance)
tts.play()

# 対話が終了状態に移るまで対話を続ける
while(dm.end == False):

    # 音声認識入力を得る	
    micStream = MicrophoneStream(RATE, CHUNK)
    asrStream = GoogleStreamingASR(RATE, micStream)
    print('<<<please speak>>>')
    result_asr = asrStream.get_asr_result()

    if hasattr(result_asr, 'alternatives') == False:
        print('Invalid ASR input')
        continue

    result_asr_utterance = result_asr.alternatives[0].transcript
    print("ユーザ： " + result_asr_utterance)
    
    # 言語理解
    result_slu = slu_parser.extract_slot_weather(result_asr_utterance)
    print(result_slu)
    
    # 対話管理へ入力
    system_utterance = dm.enter(result_slu)
    tts.generate(system_utterance)
    print("システム： " + system_utterance)
    tts.play()
    
    print()


システム： こんにちは。天気案内システムです。どの地域の天気を聞きたいですか。
<<<please speak>>>
音声パワー： 53.0[dB] 途中結果: 東京の天気を教えてくださいユーザ： 東京の天気を教えてください
東京	B-place
の	O
天気	O
を	O
教え	O
て	O
ください	O
[{'intent': '', 'slot_name': 'place', 'slot_value': '東京'}]
システム： いつの天気を聞きたいですか。

<<<please speak>>>
音声パワー： 50.4[dB] ユーザ： 明日でお願いしますす
明日	B-when
で	O
お願い	O
し	O
ます	O
[{'intent': '', 'slot_name': 'when', 'slot_value': '明日'}]
システム： ご案内します。

